In [23]:
# Code taken from here : http://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html

import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [24]:
from config import domainData
from config import num_classes as NUM_CLASSES
from wdStackDomain_alexnet_MADA import WDDomain
from logger import Logger
import torch
import torch.nn as nn
import torch.autograd as autograd
import torch.nn.init as init
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torch.autograd import Function
import torch.nn.functional as F
import numpy as np
import logit
import torchvision
from torchvision import datasets, models, transforms
import time
import random
import copy
import datetime
import itertools
import utils

In [25]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
#         transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [26]:
use_gpu = True and torch.cuda.is_available()
train_dir = domainData['amazon'] # 'amazon', 'dslr', 'webcam'
val_dir = domainData['webcam']
num_classes = NUM_CLASSES['office']
wd_param = 1.
gp_lambda = 10
num_iters = 50 # total number of training iterations
base_lr = 1e-5
EPOCHS = 50
weight_decay = 1e-7
g_loss_param = 1.
batch_size = 32 # batch_size for each of source and target samples

In [27]:
print("use gpu: ", use_gpu)

torch.manual_seed(7)
if use_gpu:
    torch.cuda.manual_seed(7)

use gpu:  True


In [28]:
image_datasets = {'train' : datasets.ImageFolder(train_dir,
                                          data_transforms['train']),
                  'val' : datasets.ImageFolder(val_dir,
                                          data_transforms['val'])
                 }
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size,
                                             shuffle=True, num_workers=4, drop_last=True)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

model_ft = WDDomain(num_classes)

if use_gpu:
    model_ft = model_ft.cuda()

In [29]:
def get_inifinite_dataloader(dataloader):
    data_iter = iter(dataloader)
    while(1):
        try:
            data = next(data_iter)
            yield data
        except StopIteration:
            data_iter = iter(dataloader)

# Using broadcasting for creating predicted label feature tensor

```python
a = torch.Tensor([[1,2,3,4,5],[10,20,30,40,50]])
b = torch.ones(2,14)

a.unsqueeze_(-1)
b.unsqueeze_(1)
print(a.size(), b.size())

print(a * b)
```

In [30]:
clscriterion = nn.CrossEntropyLoss()
# clscriterion = logit.softmax_cross_entropy_with_logits(num_classes)

param_group1 = [
# {'params' : model_ft.features.parameters(), 'lr' : 1e-4, 'betas' : (0.5, 0.9)},
{'params' : model_ft._discriminator.parameters(), 'lr': 10*base_lr, 'weight_decay' : weight_decay } # , 'lr' : 1e-4, 'betas' : (0.5, 0.9)
]
disc_opt = optim.Adam(param_group1)
# disc_opt = optim.SGD(param_group1, momentum=0.9)

param_group2 = [
{'params' : model_ft.features.net.classifier.parameters(), 'lr' : 0.001*base_lr, 'weight_decay' : weight_decay}, # , 'betas' : (0.5, 0.9
{'params' : model_ft.features.extra.parameters(), 'lr' : 10*base_lr, 'weight_decay' : weight_decay},
{'params' : model_ft.classifier.parameters(), 'lr': 10*base_lr, 'weight_decay' : weight_decay}
]
gen_opt = optim.Adam(param_group2)
# gen_opt = optim.SGD(param_group2, momentum=0.9)

# param_group3 = [
# # {'params' : model_ft.features.basenet.layer4.parameters(), 'lr' : 2e-4, 'betas' : (0.5, 0.9), 'weight_decay' : 1e-4},
# # {'params' : model_ft.features.net.classifier.parameters(), 'lr': base_lr, 'weight_decay' : 1e-5}, # , 'lr' : 1e-4, 'betas' : (0.5, 0.9)
# {'params' : model_ft.features.extra.parameters(), 'lr': 10*base_lr, 'weight_decay' : 1e-5},
# {'params' : model_ft.classifier.parameters(), 'lr': 10*base_lr, 'weight_decay' : 1e-4} # , 'lr' : 1e-4
# ]
cls_opt = None
# cls_opt = optim.Adam(param_group3)
# cls_opt = optim.SGD(param_group3, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
# tar_lr_scheduler = lr_scheduler.StepLR(taroptimizer, step_size=7, gamma=0.1)

# cls_scheduler = lr_scheduler.ExponentialLR(cls_opt, 0.1)
# gen_scheduler = lr_scheduler.ExponentialLR(gen_opt, 0.1)
# disc_scheduler = lr_scheduler.ExponentialLR(disc_opt, 0.1)

disc_sch = lr_scheduler.StepLR(disc_opt, 10, 0.1)
gen_sch = lr_scheduler.StepLR(gen_opt, 10, 0.1)

# lr_schedulers = [cls_scheduler, gen_scheduler, disc_scheduler]
lr_schedulers = [disc_sch, gen_sch]
# lr_schedulers = None

softmax = nn.Softmax(dim=1)
l2_reg = logit.L2_Loss()

In [31]:
def test_model(model_ft, criterion, save_model=False, save_name=None):
    data_iter = iter(dataloaders['val'])

    model_ft.features.eval()
    model_ft.classifier.eval()
    model_ft._discriminator.eval()
    
    acc_val = 0
    steps = 0.
    for data in data_iter:
        img, lbl = data
        if use_gpu:
            img = img.cuda()
            lbl = lbl.cuda()
        img = Variable(img, volatile=True)
        lbl = Variable(lbl, requires_grad=False)

        feat_out = model_ft.features(img)
        out = model_ft.classifier(feat_out)

        loss = criterion(out, lbl)
        
        out1 = softmax(out)
        _, preds = torch.max(out1.data, 1)
#         print("preds size: ", preds.size(), " lbl data size: ", lbl.data.size())
        acc_val += torch.eq(preds, lbl.data).float().mean()
        steps = steps + 1
    # acc = acc_val / dataset_sizes['val']
    acc = acc_val / steps
    print("validation accuracy: {:.4f}".format(acc))
    if save_model:
        torch.save(model_ft.state_dict(), save_name)
    return

In [32]:
def train_model(model, clscriterion, disc_opt, gen_opt, cls_opt=None, lr_schedulers=None, num_epochs=25):
    since = time.time()
   
    for _ in range(num_iters):
        src_data = iter(dataloaders['train'])
        tar_data = get_inifinite_dataloader(dataloaders['val'])
        running_clsacc, running_clsloss = 0., 0.
        running_critic_loss, running_gen_loss, running_gp_loss, steps = 0., 0., 0., 0

        for srcinps, srclbls in src_data:
            tarinps, _ = next(tar_data)

            srcinps = srcinps.cuda() if use_gpu else srcinps
            srclbls = srclbls.cuda() if use_gpu else srclbls
            tarinps = tarinps.cuda() if use_gpu else tarinps
            srcinps, srclbls = Variable(srcinps), Variable(srclbls)
            tarinps = Variable(tarinps)

            inps = torch.cat([srcinps, tarinps])
            feats = model.features(inps) # feats size: [batch_size, 256]

            cls_out = model.classifier(feats)
            cls_out_src = cls_out[:batch_size]
            cls_out_tar = cls_out[batch_size:]

            cls_loss = clscriterion(cls_out_src, srclbls)
            _, preds = torch.max(softmax(cls_out_src).data, 1)
            running_clsacc += torch.eq(preds, srclbls.data).float().mean()
            running_clsloss += cls_loss.data[0]

            cls_out_tar = softmax(cls_out_tar)
            assert cls_out_tar.size(1)==num_classes, "expected %d but got %s"%(num_classes, str(cls_out_tar.size()))

            srclbls_onehot = utils.one_hot_encode(srclbls, num_classes)
            assert srclbls_onehot.size()==cls_out_tar.size(), "expected same, got %s %s"%(str(srclbls_onehot.size(), cls_out_tar.size()))

            mode_wts = torch.cat([srclbls_onehot, cls_out_tar.data]) # size: [batch, num_classes]
            mode_wts.unsqueeze_(-1) # size: [batch, num_classes, 1]
            mode_wts = Variable(mode_wts, requires_grad=False)
            
            critic_in = feats.unsqueeze(1) # size: [batch, 1, 256]            
            critic_in = critic_in * mode_wts # critic_in size: [batch_size, num_classes, 256]

            critic_out = model._discriminator(critic_in)
            D = critic_out[:batch_size]
            D_ = critic_out[batch_size:]
            critic_loss = (torch.sigmoid(D_) ** 2).mean() + ((1. - torch.sigmoid(D)) ** 2).mean()
            gen_loss = D.mean() - D_.mean()

            running_critic_loss += critic_loss.data[0]
            running_gen_loss += gen_loss.data[0]

            feats_wt_src = critic_in[:batch_size].data
            feats_wt_tar = critic_in[batch_size:].data

            alpha = torch.Tensor(batch_size, 1).uniform_(0,1).unsqueeze_(-1)
            alpha = alpha.cuda() if use_gpu else alpha
            diff = feats_wt_src - feats_wt_tar
            interpolates = feats_wt_src + (alpha * diff)
            interpolates = Variable(interpolates, requires_grad=True)

            inter_out = model._discriminator(interpolates)
            ones = torch.ones(inter_out.size())
            ones = ones.cuda() if use_gpu else ones

            grads = autograd.grad(inter_out, interpolates, grad_outputs=ones,
                retain_graph=True, create_graph=True, only_inputs=False)[0]
            gp = ((grads.norm(2, dim=1) - 1) ** 2).mean()

            running_gp_loss += gp.data[0]

            for params in model._discriminator.parameters():
                params.requires_grad=True

            critic_loss_t = critic_loss * wd_param + gp * gp_lambda
            disc_opt.zero_grad()
            critic_loss_t.backward(retain_graph=True)
            disc_opt.step()

            for params in model._discriminator.parameters():
                params.requires_grad=False

            total_loss = cls_loss + gen_loss * g_loss_param

            gen_opt.zero_grad()
            total_loss.backward()
            gen_opt.step()
            steps = steps + 1
            
        print("critic loss: {:.4f}, gen loss: {:.4f}, gp loss: {:.4f}".
              format(running_critic_loss/steps, running_gen_loss/steps, running_gp_loss/steps))
        print("classification loss: {:.4f}, classification accuracy: {:.4f}".
             format(running_clsloss/steps, running_clsacc/steps))

        if lr_schedulers:
            for x in lr_schedulers:
                x.step()

        test_model(model, clscriterion, False, None)

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))    
    return

```python
a = torch.Tensor(32,31,256).uniform_(0,1)

conv1 = nn.Sequential(nn.Conv1d(31,31,70), nn.Conv1d(31,31,130), nn.Conv1d(31,16,58), nn.Conv1d(16,1,1))

b = conv1(Variable(a))

b.size()
```

In [33]:
train_model(model_ft, clscriterion, disc_opt, gen_opt, cls_opt, lr_schedulers, num_epochs=EPOCHS)

save_name = "grl_model_with_transform.pth"
test_model(model_ft, clscriterion, False, save_name)
test_model(model_ft, clscriterion, False, save_name)
test_model(model_ft, clscriterion, False, save_name)
test_model(model_ft, clscriterion, False, save_name)
test_model(model_ft, clscriterion, False, save_name)

critic loss: 0.5000, gen loss: 0.0075, gp loss: 0.9971
classification loss: 1.7858, classification accuracy: 0.5781
validation accuracy: 0.4779
critic loss: 0.5004, gen loss: 0.0063, gp loss: 0.9971
classification loss: 0.8669, classification accuracy: 0.7887
validation accuracy: 0.5000
critic loss: 0.5009, gen loss: 0.0044, gp loss: 0.9971
classification loss: 0.6483, classification accuracy: 0.8391
validation accuracy: 0.4961
critic loss: 0.5013, gen loss: 0.0029, gp loss: 0.9970
classification loss: 0.5147, classification accuracy: 0.8764
validation accuracy: 0.5091
critic loss: 0.5017, gen loss: 0.0013, gp loss: 0.9970
classification loss: 0.4181, classification accuracy: 0.8956
validation accuracy: 0.4909
critic loss: 0.5022, gen loss: -0.0004, gp loss: 0.9970
classification loss: 0.3373, classification accuracy: 0.9293
validation accuracy: 0.5039
critic loss: 0.5026, gen loss: -0.0022, gp loss: 0.9969
classification loss: 0.2768, classification accuracy: 0.9457
validation accurac